In [1]:
from AgentSimulate import Agent
from game.PokerSimple import PokerSimple
import random
import numpy as np
import copy
import tensorflow_probability as tfp
import tensorflow as tf

2022-05-25 14:11:50.430318: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-25 14:11:50.430344: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
game = PokerSimple(0,1)

In [3]:
agent = Agent()

2022-05-25 14:11:53.243427: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-05-25 14:11:53.243472: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-25 14:11:53.243512: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (cromi-Lenovo-V15-ADA): /proc/driver/nvidia/version does not exist
2022-05-25 14:11:53.244735: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
import random
class MCST:
    def __init__(self, gamestate, agent, perspective, n_runs = 10, n_sim = 10):
        self.policies = []
        self.observations = []
        self.game = copy.deepcopy(gamestate)
        if self.game.street == 0:
            deck = self.game.deck[1:]
            random.shuffle(deck)
            self.game.deck[1:]=deck
            self.game.hole_1=self.game.deck[1]

        else:
            board = self.game.board
            deck = []
            deck.append(self.game.deck[1])
            deck.extend(self.game.deck[3:])
            random.shuffle(deck)
            self.game.deck[1]=deck[0]
            self.game.deck[3:]= deck[1:]
            self.game.hole_1=self.game.deck[1]
        self.game.run_ranges = True
        self.agent = agent
        self.perspective = perspective
        self.n_runs = n_runs
        self.n_sim = n_sim

    def search(self):
        game = self.game
        agent = self.agent
        actions_taken = {0:0, 1:0, 2:0} #to count number of actions taken in simulations
        evs = {} #save evs for each action

        #find legal actions in state
        if game.stacks[0] == 4.5 or game.stacks[1]==4.5: #beginning of round
            legal_actions = [0,1,2]
        elif game.stacks[0] != game.stacks[1]:
            legal_actions = [0,1]
        else:
            legal_actions = [1,2]

        #initialize policy
        game.create_observation(game.next_to_act[0])
        observation = game.observations[game.next_to_act[0]][:]
        self.observation = observation[:]
        if observation[4] == observation[6]:
            observation.append(1)
        else:
            observation.append(0)

        state = tf.convert_to_tensor([observation])
        policy = np.array(agent.actor(state))[0]

        for i in range(self.n_runs):

            action_probabilities = tfp.distributions.Categorical(probs=policy)

            action = int(np.array(action_probabilities.sample()))
            while action not in legal_actions:
                action = int(np.array(action_probabilities.sample()))

            actions_taken[action] += 1
            if action not in evs.keys():
                evs[action] = 0
                saw_flop = agent.saw_flop[:]
                ranges = agent.ranges_vil[:]
                reward_sim = 0
                for j in range(self.n_sim):
                    game_sim = copy.deepcopy(game)
                    #add ranges villain
                    game_sim.ranges[1-self.perspective] = agent.ranges_vil[self.perspective]

                    game_sim.implement_action(game_sim.next_to_act[0],action)
                    if (not agent.saw_flop[1-self.perspective]) and game_sim.street ==1:
                        hole_hero = game_sim.hole_0 if self.perspective == 0 else game_sim.hole_1
                        agent.update_range_flop(self.perspective, hole_hero, game_sim.board)
                        game_sim.ranges[1-self.perspective] = agent.ranges_vil[self.perspective]

                    if game_sim.done:
                        reward = game_sim.stacks[self.perspective] -4.5 if game_sim.observations[self.perspective][0] == 0 else game_sim.stacks[self.perspective] - 4
                    else:
                        while (not game_sim.done) and game_sim.next_to_act[0]!=self.perspective: #villain acts
                            game_sim.create_observation(1-self.perspective)
                            observation = game_sim.observations[1-self.perspective][:]
                            observation[4] =0
                            observation.append(-1)
                            states = np.array([observation for obs in range(5)])
                            states[:,4] = [card for card in range(1,6)]
                            states[:,-1] = (states[:,4]==states[:,6]).astype(int)
                            policy_villain = np.array(agent.actor(states))
                            new_policy_vil = (np.expand_dims(agent.ranges_vil[self.perspective],axis=1)*policy_villain).sum(axis=0)
                            action_vil = random.choices([0,1,2], weights = new_policy_vil)[0]
                            agent.update_range_action(self.perspective, observation[:-1], action_vil)
                            game_sim.ranges[1-self.perspective] = agent.ranges_vil[self.perspective]
                            game_sim.implement_action(1-self.perspective,action_vil)
                    if game_sim.done:
                        position_hero = game_sim.position_0 if self.perspective == 0 else game_sim.position_1
                        reward = game_sim.stacks[self.perspective] -4.5 if position_hero == 0 else game_sim.stacks[self.perspective] - 4
                    else:
                        new_tree = MCST(game_sim,  agent, self.perspective,n_runs = 50, n_sim = 50)
                        evs_tree=new_tree.search()[0]
                        reward = max(evs_tree, key=evs_tree.get)
                    reward_sim += reward
                #reset pre-simulation ranges
                agent.ranges_vil=ranges
                agent.saw_flop=saw_flop



                evs[action]+=reward_sim/self.n_sim
            for act in range(3):
                if act not in legal_actions:
                    if policy[act]> 0.01:
                        policy = self.update_policy(policy, act, -5)

            policy = self.update_policy(policy, action, evs[action])
            if policy[0]>0.999 or policy[1] > 0.999 or policy[2]>0.999:
                break

        agent.store_sample(self.observation, policy)
        return evs, policy, actions_taken

    def update_policy(self, old_policy, action, reward):
        logit_action = np.log(old_policy[action]/(1-old_policy[action]))
        logit_action += 0.05*reward/old_policy[action]
        prob_action_new = np.exp(logit_action)/(1+np.exp(logit_action))
        new_policy = [element/(1-old_policy[action])*(1-prob_action_new) for element in old_policy]
        new_policy[action] = prob_action_new
        return np.array(new_policy)

In [5]:
agent_0 = Agent()
agent_1 = Agent()
game=PokerSimple(0,1)

In [6]:
games = []
observations = []

In [7]:
game.reset()

(False, 1, [0, 4.5, 4, 0, 1, 1.5, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1])

In [101]:
games = []
observations = []
import copy


for card in range(1,6):
    done, player, obs = game.reset()
    while player!= 0 or obs[4] != card:
        done, player, obs = game.reset()
    games.append(copy.deepcopy(game))
    observations.append(obs)

for card in range(1,6):
    done, player, obs = game.reset()
    while player != 0 or game.hole_0 != card:
        done, player, obs = game.reset()

    game.implement_action(0,1)
    game.implement_action(1,2)
    games.append(copy.deepcopy(game))
    game.create_observation(0)
    observations.append(game.observations[0][:])

for card in range(1,6):
    done, player, obs = game.reset()
    while player!= 1 or game.hole_0 != card:
        done, player, obs = game.reset()
    game.implement_action(1,1)
    game.create_observation(0)
    games.append(copy.deepcopy(game))
    observations.append(game.observations[0][:])


for card in range(1,6):
    done, player, obs = game.reset()
    while player!= 1 or game.hole_0 != card:
        done, player, obs = game.reset()

    game.implement_action(1,2)
    game.create_observation(0)
    games.append(copy.deepcopy(game))
    observations.append(game.observations[0][:])

#postflop - sb to check

for card in range(1,6):
    for board in range(1,6):
        done, player, obs = game.reset()
        while game.hole_0 != card or game.deck[2]!=board or player ==1:
            done, player, obs = game.reset()

        game.implement_action(0,1)
        game.implement_action(1,1)
        game.implement_action(1,1)
        games.append(copy.deepcopy(game))
        game.create_observation(0)
        observations.append(game.observations[0][:])

#postflop sb to push
for card in range(1,6):
    for board in range(1,6):
        done, player, obs = game.reset()
        while game.hole_0 != card or game.deck[2]!=board or player ==1:
            done, player, obs = game.reset()

        game.implement_action(0,1)
        game.implement_action(1,1)
        game.implement_action(1,2)
        games.append(copy.deepcopy(game))
        game.create_observation(0)
        observations.append(game.observations[0][:])

#postflop bb first in
for card in range(1,6):
    for board in range(1,6):
        done, player, obs = game.reset()
        while game.hole_0 != card or game.deck[2]!=board or player !=1:
            done, player, obs = game.reset()

        game.implement_action(1,1)
        game.implement_action(0,1)
        games.append(copy.deepcopy(game))
        game.create_observation(0)
        observations.append(game.observations[0][:])

#postflop bb to push
for card in range(1,6):
    for board in range(1,6):
        done, player, obs = game.reset()
        while game.hole_0 != card or game.deck[2]!=board or player !=1:
            done, player, obs = game.reset()

        game.implement_action(1,1)
        game.implement_action(0,1)
        game.implement_action(0,1)
        game.implement_action(1,2)
        games.append(copy.deepcopy(game))
        game.create_observation(0)
        observations.append(game.observations[0][:])

In [ ]:

for ep in range(1,100000):
    print(ep)
    game.reset()
    agent_1.actor = agent_0.actor
    agent_0.set_range_vil(0,game.hole_0)
    agent_1.set_range_vil(1,game.hole_1)
    agent_0.saw_flop = [False, False]
    agent_1.saw_flop = [False, False]
    while not game.done:
        if game.next_to_act[0]== 0:
            tree = MCST(game,agent_0, 0, n_runs = 50, n_sim = 50)
            evs, policy, _ = tree.search()

            action = max(evs, key=evs.get)
            game.create_observation(0)
            state = game.observations[0][:]
            agent_0.store_sample(state,policy)

            obs_vil = game.observations[0][:]
            obs_vil[4]=0
            agent_1.update_range_action(1,obs_vil, action)
            game.implement_action(0, action)
            if (not agent_0.saw_flop[1]) and game.street == 1:
                agent_0.update_range_flop(0, game.hole_0, game.board)


        else:
            tree = MCST(game,agent_1, 1, n_runs = 50, n_sim = 50)
            evs, policy, _ = tree.search()
            action = max(evs, key=evs.get)
            game.create_observation(1)
            state = game.observations[1][:]
            agent_0.store_sample(state,policy)
            obs_vil = game.observations[1][:]
            obs_vil[4]=0
            agent_0.update_range_action(0,obs_vil, action)
            game.implement_action(1, action)
            if (not agent_1.saw_flop[0]) and game.street == 1:
                agent_1.update_range_flop(1, game.hole_1, game.board)


        if ep%50 == 0:
            agent_0.print_strategy()



1
2
3


In [25]:
agent_0.memory.policy_memory

array([[4.9483147e-04, 4.0511883e-04, 9.9910003e-01],
       [9.9703640e-01, 2.9637804e-03, 3.4688523e-09],
       [9.9703640e-01, 2.9637804e-03, 3.4688523e-09],
       ...,
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00]], dtype=float32)

In [23]:
agent_0.memory.state_memory

array([[ 0. ,  4.5,  4. , ..., -1. , -1. ,  0. ],
       [ 1. ,  4. ,  0. , ..., -1. , -1. ,  0. ],
       [ 1. ,  4. ,  0. , ..., -1. , -1. ,  0. ],
       ...,
       [ 0. ,  0. ,  0. , ...,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. , ...,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. , ...,  0. ,  0. ,  0. ]], dtype=float32)

In [8]:

agent_1.actor = agent_0.actor
agent_0.set_range_vil(0,game.hole_0)
agent_1.set_range_vil(1,game.hole_1)
agent_0.saw_flop = [False, False]
agent_1.saw_flop = [False, False]
game.reset()

(False, 1, [0, 4.5, 4, 0, 2, 1.5, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1])

In [9]:
tree = MCST(game,agent_1, 1, n_runs = 50, n_sim = 50)
evs, policy, _ = tree.search()

create new tree
new tree policies [array([2.16961716e-07, 2.41972077e-02, 9.75802576e-01])]
new tree obs [[0, 4, 4, 1, 4, 2, 5, 0.5, 0, 0, -1, -1, -1, -1, -1, -1, -1, 0]]
new tree policy1 [2.16961716e-07 2.41972077e-02 9.75802576e-01]
new tree obs1 [0, 4, 4, 1, 4, 2, 5, 0.5, 0, 0, -1, -1, -1, -1, -1, -1, -1, 0]
create new tree
new tree policies [array([0.00404267, 0.9789968 , 0.0169607 ])]
new tree obs [[0, 4, 4, 1, 1, 2, 5, 0.5, 0, 0, -1, -1, -1, -1, -1, -1, -1, 0]]
new tree policy1 [0.00404267 0.9789968  0.0169607 ]
new tree obs1 [0, 4, 4, 1, 1, 2, 5, 0.5, 0, 0, -1, -1, -1, -1, -1, -1, -1, 0]
create new tree
new tree policies [array([0.92466915, 0.07168951, 0.00364133])]
new tree obs [[0, 4, 0, 0, 2, 6, 0, 0.5, 4, -1, -1, -1, -1, -1, -1, -1, -1, 0]]
new tree policy1 [0.92466915 0.07168951 0.00364133]
new tree obs1 [0, 4, 0, 0, 2, 6, 0, 0.5, 4, -1, -1, -1, -1, -1, -1, -1, -1, 0]
create new tree
new tree policies [array([8.23755724e-04, 9.73568265e-01, 2.56080362e-02])]
new tree obs [[

In [13]:
tree.observations

[[0, 4, 0, 0, 4, 6, 0, 0.5, 4, -1, -1, -1, -1, -1, -1, -1, -1, 0]]

In [7]:
X = agent_0.memory.state_memory[:agent_0.memory.mem_cntr]
y = agent_0.memory.policy_memory[:agent_0.memory.mem_cntr]

In [10]:
from keras import backend as K
K.set_value(agent_0.actor.optimizer.learning_rate, 0.000001)
agent_0.actor.fit(X,y, batch_size= 4, epochs = 1000)

Epoch 1/1000
2286/2286 [==============================] - 7s 3ms/step - loss: 0.2187
Epoch 2/1000
2286/2286 [==============================] - 7s 3ms/step - loss: 0.2187
Epoch 3/1000
2286/2286 [==============================] - 7s 3ms/step - loss: 0.2187
Epoch 4/1000
2286/2286 [==============================] - 8s 3ms/step - loss: 0.2187
Epoch 5/1000
2286/2286 [==============================] - 7s 3ms/step - loss: 0.2187
Epoch 6/1000
2286/2286 [==============================] - 7s 3ms/step - loss: 0.2187
Epoch 7/1000
2286/2286 [==============================] - 8s 3ms/step - loss: 0.2186
Epoch 8/1000
2286/2286 [==============================] - 7s 3ms/step - loss: 0.2186
Epoch 9/1000
2286/2286 [==============================] - 7s 3ms/step - loss: 0.2185
Epoch 10/1000
2286/2286 [==============================] - 8s 3ms/step - loss: 0.2185
Epoch 11/1000
2286/2286 [==============================] - 8s 3ms/step - loss: 0.2187
Epoch 12/1000
2286/2286 [==============================] - 8s 3

KeyboardInterrupt: 

In [11]:
agent_0.print_strategy()

--------------------------------------
preflop

sb
 first in

holecard 1 probs [0.002, 0.717, 0.281]
holecard 2 probs [0.0, 0.677, 0.323]
holecard 3 probs [0.0, 0.593, 0.406]
holecard 4 probs [0.0, 0.298, 0.702]
holecard 5 probs [0.0, 0.112, 0.888]
preflop

sb
 complete - push

holecard 1 probs [0.991, 0.009, 0.0]
holecard 2 probs [0.98, 0.02, 0.0]
holecard 3 probs [0.302, 0.697, 0.002]
holecard 4 probs [0.002, 0.996, 0.002]
holecard 5 probs [0.0, 0.998, 0.002]

bb
  to complete

holecard 1 probs [0.001, 0.706, 0.293]
holecard 2 probs [0.0, 0.716, 0.284]
holecard 3 probs [0.0, 0.579, 0.421]
holecard 4 probs [0.0, 0.354, 0.646]
holecard 5 probs [0.0, 0.254, 0.746]

bb
  to push

holecard 1 probs [0.997, 0.003, 0.0]
holecard 2 probs [0.993, 0.007, 0.0]
holecard 3 probs [0.435, 0.565, 0.0]
holecard 4 probs [0.001, 0.998, 0.0]
holecard 5 probs [0.0, 0.999, 0.001]
--------------------------------------
postflop

sb
  to check

board 1
holecard 1 probs [0.0, 0.037, 0.963]
holecard 2 probs [0

In [244]:
agent_0.saw_flop = [False, False]
agent_0.set_range_vil(0, 3)
tree = MCST(game,agent_0,0, n_runs=100, n_sim=20)
tree.search()

[1.1927022e-04 8.2748395e-01 1.7239676e-01]
[8.7539784e-06 6.2499702e-01 3.7499428e-01]
{2: 1.5, 1: 1.4150000000000003}
[8.7539784e-06 6.2499702e-01 3.7499428e-01]
{1: 1.4400000000000002, 2: 1.5}
[8.7539784e-06 6.2499702e-01 3.7499428e-01]
{2: 1.5, 1: 1.5}
[8.7539784e-06 6.2499702e-01 3.7499428e-01]
{1: 1.5, 2: 1.5}
[2.588930e-02 9.740409e-01 6.981559e-05]
{1: 5.375}
[8.7539784e-06 6.2499702e-01 3.7499428e-01]
{2: 1.9, 1: 1.49}
[2.588930e-02 9.740409e-01 6.981559e-05]
{1: 5.375}
[2.588930e-02 9.740409e-01 6.981559e-05]
{1: 5.0}
[2.588930e-02 9.740409e-01 6.981559e-05]
{1: 4.625}
[8.7539784e-06 6.2499702e-01 3.7499428e-01]
{1: 1.385, 2: 2.3}
[8.7539784e-06 6.2499702e-01 3.7499428e-01]
{1: 1.475, 2: 1.825}
[8.7539784e-06 6.2499702e-01 3.7499428e-01]
{2: 3.7, 1: 1.49}
[8.7539784e-06 6.2499702e-01 3.7499428e-01]
{1: 1.4949999999999999, 2: 2.7}
[8.7539784e-06 6.2499702e-01 3.7499428e-01]
{1: 1.5, 2: 2.7}
[8.7539784e-06 6.2499702e-01 3.7499428e-01]
{1: 1.4949999999999999, 2: 2.7}
[2.588930e-

({1: 1.7, 2: -2.846875},
 array([2.55957817e-05, 9.90664778e-01, 9.30962302e-03]),
 {0: 0, 1: 19, 2: 1})

In [30]:
game.ranges[1-tree.perspective] = agent_0.ranges_vil[tree.perspective]

In [31]:
game.ranges

{0: None,
 1: [0.2222222222222222,
  0.2222222222222222,
  0.2222222222222222,
  0.2222222222222222,
  0.1111111111111111]}

In [11]:
game.ranges

{0: None, 1: None}

In [37]:
agent_0.saw_flop = [False, False]
agent_0.set_range_vil(1, 2)
tree = MCST(game,agent_0,1, n_runs=100, n_sim=100)
tree.search()

({1: 0.8, 2: 0.9175, 0: 0.0},
 array([0.08801841, 0.31111797, 0.60086362]),
 {0: 25, 1: 28, 2: 47})

In [12]:
import pickle
import numpy as np

with open('q_simple', 'rb') as q_simple_file:
    q = pickle.load(q_simple_file)
def choose_action_q(observation):

    hole = observation[4]
    #sb:
    if observation[0] == 0:
        if observation[3] == 0: #preflop
            if observation[-10] ==-1: #first in
                action = np.argmax(q['sb'][hole]['preflop']['preflop_fi'])

            else: # bb must have gone allin, if sb has another action to perform
                action = np.argmax(q['sb'][hole]['preflop']['preflop_to_push'])


        else: #postflop
            board = observation[6]
            if observation[2] != 0: #bb did not push
                action = np.argmax(q['sb'][hole]['postflop'][board]['to_check'])
                if action == 0:
                    action =1
                else:
                    action =2
            else:
                action = np.argmax(q['sb'][hole]['postflop'][board]['to_push'])


    #bb
    else:
        if observation[3] == 0: #preflop
            if observation[-10] == 0.5: #sb completed
                action = np.argmax(q['bb'][hole]['preflop']['to_call'])
                if action == 0:
                    action =1
                else:
                    action =2
            else: #sb pushed
                action = np.argmax(q['bb'][hole]['preflop']['to_push'])

        else:
            board = observation[6]
            if observation[2]!= 0: #bb is first in
                action = np.argmax(q['bb'][hole]['postflop'][board]['postflop_fi'])
                if action == 0:
                    action =1
                else:
                    action =2
            else: #bb checked and sb pushed
                action = np.argmax(q['bb'][hole]['postflop'][board]['postflop_to_push'])

    return action

In [34]:

n_games = 1000
total_chips = 0
for ep in range(n_games):
    print(ep)
    game.reset()
    agent_0.set_range_vil(0,game.hole_0)
    agent_0.saw_flop = [False, False]

    while not game.done:
        if game.next_to_act[0]== 0:
            game_sim = copy.deepcopy(game)
            if game_sim.street == 0:
                deck = game_sim.deck[1:]
                random.shuffle(deck)
                game_sim.deck[1:]=deck
                game_sim.hole_1=game_sim.deck[1]

            else:
                board = game_sim.board
                deck = []
                deck.append(game_sim.deck[1])
                deck.extend(game_sim.deck[3:])
                random.shuffle(deck)
                game_sim.deck[1]=deck[0]
                game_sim.deck[3:]= deck[1:]
                game_sim.hole_1=game_sim.deck[1]
            tree = MCST(game_sim,agent_0, 0, n_runs = 25, n_sim = 25)
            evs, policy, _ = tree.search()
            action = max(evs, key=evs.get)
            game.implement_action(0, action)
            if (not agent_0.saw_flop[1]) and game.street == 1:
                agent_0.update_range_flop(0, game.hole_0, game.board)


        else:
            game.create_observation(1)
            action = choose_action_q(game.observations[1])
            obs_vil = game.observations[1][:]
            obs_vil[4] = 0
            agent_0.update_range_action(0,obs_vil,action)
            if (not agent_0.saw_flop[1]) and game.street == 1:
                agent_0.update_range_flop(0, game.hole_0, game.board)
            game.implement_action(1, action)

    result = game.stacks[0] - 5
    total_chips += result
    avg_chips = total_chips/(ep+0.00000000001)
    print("total", total_chips, "avg", avg_chips)


0
total 1.0 avg 100000000000.0
1
total -2.75 avg -2.7499999999725
2
total -6.5 avg -3.24999999998375
3
total -10.25 avg -3.416666666655278
4
total -9.25 avg -2.312499999994219
5
total -13.0 avg -2.5999999999948
6
total -18.0 avg -2.999999999995
7
total -17.0 avg -2.428571428567959
8
total -20.75 avg -2.593749999996758
9
total -25.75 avg -2.8611111111079324
10
total -22.0 avg -2.1999999999978
11
total -18.25 avg -1.659090909089401
12
total -18.25 avg -1.520833333332066
13
total -17.25 avg -1.3269230769220564
14
total -18.25 avg -1.3035714285704976
15
total -18.25 avg -1.2166666666658557
16
total -17.25 avg -1.078124999999326
17
total -16.25 avg -0.9558823529406142
18
total -16.25 avg -0.9027777777772762
19
total -15.25 avg -0.8026315789469459
20
total -11.5 avg -0.5749999999997125
21
total -10.5 avg -0.49999999999976186
22
total -9.5 avg -0.43181818181798554
23
total -8.5 avg -0.3695652173911437
24
total -12.25 avg -0.510416666666454
25
total -16.0 avg -0.639999999999744
26
total -19.75

KeyboardInterrupt: 

In [ ]:
n_games = 10000
total_chips = 0
for ep in range(n_games):
    print(ep)
    game.reset()
    agent_0.set_range_vil(0,game.hole_0)
    agent_0.saw_flop = [False, False]

    while not game.done:

        if game.next_to_act[0]== 0:
            game_sim = copy.deepcopy(game)
            if game_sim.street == 0:
                deck = game_sim.deck[1:]
                random.shuffle(deck)
                game_sim.deck[1:]=deck
                game_sim.hole_1=game_sim.deck[1]

            else:
                board = game_sim.board
                deck = []
                deck.append(game_sim.deck[1])
                deck.extend(game_sim.deck[3:])
                random.shuffle(deck)
                game_sim.deck[1]=deck[0]
                game_sim.deck[3:]= deck[1:]
                game_sim.hole_1=game_sim.deck[1]

            tree = MCST(game_sim,agent_0, 0, n_runs = 50, n_sim = 50)
            evs, policy, _ = tree.search()
            action = max(evs, key=evs.get)
            game.implement_action(0, action)
            if (not agent_0.saw_flop[1]) and game.street == 1:
                agent_0.update_range_flop(0, game.hole_0, game.board)


        else:
            game.create_observation(1)
            action = choose_action_q(game.observations[1])
            obs_vil = game.observations[1][:]
            obs_vil[4] = 0
            agent_0.update_range_action(0,obs_vil,action)
            if (not agent_0.saw_flop[1]) and game.street == 1:
                agent_0.update_range_flop(0, game.hole_0, game.board)
            game.implement_action(1, action)

    result = game.stacks[0] - 5
    total_chips += result
    avg_chips = total_chips/(ep+0.00000000001)
    print("total", total_chips, "avg", avg_chips)

0
total 1.0 avg 100000000000.0
1
total 2.0 avg 1.99999999998
2
total 3.0 avg 1.4999999999925
3
total 2.0 avg 0.6666666666644444
4
total 1.0 avg 0.249999999999375
5
total 2.0 avg 0.3999999999992
6
total 5.75 avg 0.9583333333317361
7
total 6.75 avg 0.9642857142843367
8
total 7.75 avg 0.9687499999987892
9
total 8.75 avg 0.9722222222211421
10
total 9.75 avg 0.9749999999990251
11


In [44]:
agent_0.print_strategy()

--------------------------------------
preflop

sb
 first in

holecard 1 probs [0.002, 0.717, 0.281]
holecard 2 probs [0.0, 0.677, 0.323]
holecard 3 probs [0.0, 0.593, 0.406]
holecard 4 probs [0.0, 0.298, 0.702]
holecard 5 probs [0.0, 0.112, 0.888]
preflop

sb
 complete - push

holecard 1 probs [0.991, 0.009, 0.0]
holecard 2 probs [0.98, 0.02, 0.0]
holecard 3 probs [0.302, 0.697, 0.002]
holecard 4 probs [0.002, 0.996, 0.002]
holecard 5 probs [0.0, 0.998, 0.002]

bb
  to complete

holecard 1 probs [0.001, 0.706, 0.293]
holecard 2 probs [0.0, 0.716, 0.284]
holecard 3 probs [0.0, 0.579, 0.421]
holecard 4 probs [0.0, 0.354, 0.646]
holecard 5 probs [0.0, 0.254, 0.746]

bb
  to push

holecard 1 probs [0.997, 0.003, 0.0]
holecard 2 probs [0.993, 0.007, 0.0]
holecard 3 probs [0.435, 0.565, 0.0]
holecard 4 probs [0.001, 0.998, 0.0]
holecard 5 probs [0.0, 0.999, 0.001]
--------------------------------------
postflop

sb
  to check

board 1
holecard 1 probs [0.0, 0.037, 0.963]
holecard 2 probs [0

In [16]:
n_games = 10000
total_chips = 0
for ep in range(n_games):
    print(ep)
    game.reset()

    while not game.done:
        if game.next_to_act[0]==0:
            game.create_observation(0)
            action = agent_0.choose_action(game.observations[0])
            game.implement_action(0,action)
        else:
            game.create_observation(1)
            action = choose_action_q(game.observations[1])
            game.implement_action(1,action)
    result = game.stacks[0] - 5
    total_chips += result
    avg_chips = total_chips/(ep+0.00000000001)
    print("total", total_chips, "avg", avg_chips)


0
total 1 avg 100000000000.0
1
total -2.75 avg -2.7499999999725
2
total -7.75 avg -3.874999999980625
3
total -6.75 avg -2.2499999999925
4
total -5.75 avg -1.4374999999964062
5
total -4.75 avg -0.9499999999981
6
total -3.75 avg -0.6249999999989584
7
total -2.75 avg -0.39285714285658163
8
total -2.75 avg -0.34374999999957034
9
total -1.75 avg -0.19444444444422843
10
total -0.75 avg -0.074999999999925
11
total 0.25 avg 0.022727272727252067
12
total -4.75 avg -0.3958333333330035
13
total -3.75 avg -0.2884615384613166
14
total -4.75 avg -0.33928571428547194
15
total -4.75 avg -0.3166666666664556
16
total -8.5 avg -0.5312499999996679
17
total -7.5 avg -0.44117647058797577
18
total -6.5 avg -0.3611111111109105
19
total -11.5 avg -0.6052631578944182
20
total -10.5 avg -0.5249999999997375
21
total -9.5 avg -0.45238095238073694
22
total -13.25 avg -0.6022727272724535
23
total -13.25 avg -0.5760869565214887
24
total -12.25 avg -0.510416666666454
25
total -8.5 avg -0.33999999999986397
26
total -7.

KeyboardInterrupt: 

In [15]:
game.run_ranges

False